# Config

In [476]:
# Import dependencies
import pandas as pd
from datetime import datetime
from sleeperpy import User, Leagues
from ffpackage.viz import compareFranchises
import plotly
import plotly.express as px

In [477]:
# Settings
date_string = datetime.today().strftime('%Y%m%d')
weekStart = 14
weekEnd = 14
def_starting_pts = 18
pr_bonus = 0
kr_bonus = 0
username = 'dirtywizard'

# Extract

In [478]:
# Read database
preds = pd.read_csv('db/records_preds.csv')
preds = preds.loc[preds['date_prediction']==preds['date_prediction'].max()]
preds = preds.loc[(preds['week']>=weekStart)&(preds['week']<=weekEnd)]
mults = pd.read_csv('db/pts_multipliers.csv')
rostLims = pd.read_csv('db/roster_limits.csv')

In [479]:
def processRosters(league_id):
    # Get rosters for all league members
    rosters = pd.DataFrame(Leagues.get_rosters(league_id))
    # Explode the player columns to find how they are placed on the roster
    result = pd.DataFrame(columns=['owner_id', 'player_id'])
    for col in ['taxi', 'starters', 'players']:
        df = rosters[['owner_id', col]].explode(col)
        df = df.rename(columns={col:'player_id'})
        df[col] = True
        result = result.merge(df, how='outer', on=['owner_id', 'player_id'])
    return result

In [480]:
def procesOwners(league_id):
    owners = pd.DataFrame(Leagues.get_users(league_id))
    list = []
    for i in range(len(owners)):
        if 'team_name' in owners['metadata'][i]:
            list.append(owners['metadata'][i]['team_name'])
        else:
            list.append(owners['display_name'][i])
    owners['franchise_name'] = list
    owners = owners[['user_id', 'franchise_name', 'display_name']]
    return owners

In [481]:
# Get User's Sleeper ID
user_id = User.get_user(username)['user_id']
# Get the User's Sleeper league ID
league_id = Leagues.get_all_leagues(user_id, 'nfl', 2023)[0]['league_id']
# Get all players on rosters
rosters = processRosters(league_id)
# Get info on all owners
owners = procesOwners(league_id)
# Get league info
user_league = Leagues.get_league(league_id)

# Get roster positions 
rosterList = user_league['roster_positions']
# Sort rosterList
sort_order = ['QB', 'RB', 'WR', 'TE', 'K', 'DEF', 'FLEX', 'SUPER_FLEX', 'BN']
rosterList.sort(key=sort_order.index)

# Get number of franchises
number_of_franchises = user_league['total_rosters']

In [482]:
       
# # Convert raw prediction values to numeric
# colList = ['Att', 'Comp', 'Pass Yds', 'Pass TDs', 'Int', 'Sck', 'Rush', 'Rsh Yds', 'Rsh TDs', 'Fum', 'Tgt', \
#     'RZ Tgt', 'Rec', 'Rec Yds', 'Rec TDs', 'Kick Ret Yds', 'XPM', 'XPA', 'FGM', 'FGA', '10-19 FGM', '20-29 FGM', \
#         '30-39 FGM', '40-49 FGM', '50+ FGM', 'Miss', 'Yds Allowed', 'Pts Agn', 'Scks', 'DefTD', 'Safts']
# for col in colList:
#     sharks[col] = sharks[col].astype('float64')
    
# # Clean up
# sharks = sharks.rename(columns={'Week':'week'})
# # Create ID: Remove Rookie "R": format first name last name
# cond1 = sharks['Player'].str[-1]=="R"
# sharks.loc[cond1, 'Player'] = sharks.loc[cond1, 'Player'].str[:-1]
# sharks['lName'] = sharks['Player'].str.split(", ", expand=True)[0]
# sharks['fName'] = sharks['Player'].str.split(", ", expand=True)[1]
# sharks['id_sharks'] = sharks['fName']+" "+sharks['lName']
# sharks['id_sharks'] = sharks['id_sharks'].str.replace(".", "")
# sharks['id_sharks'] = sharks['id_sharks'].str.title()
# sharks['id_sharks'] = [" ".join(x.split(" ")[:2]) for x in sharks['id_sharks']]
# # Reduce
# sharks = sharks[['id_sharks', 'week']+colList]


In [483]:
# Need to bin and cut def pts allowed, def yds allowed
# Need to create column for fgm>30

In [484]:
# [
# 'Comp':'pass_cmp',
# 'Pass Yds':'pass_yd',
# 'Pass TDs':'pass_td',
# 'Int':'pass_int',
# 'Rush':'rush_att',
# 'Rsh Yds':'rush_yd',
# 'Rsh TDs':'rush_td',
# 'Fum':'fum''fum_lost',
# 'Rec':'rec',
# 'Rec Yds':'rec_yd',
# 'Rec TDs':'rec_td',
# 'Kick Ret Yds':'kr_yd',
# 'XPM':'xpm',
# 'FGM':'fgm',
# '10-19 FGM':'fgm_0_19',
# '20-29 FGM':'fgm_20_29',
# '30-39 FGM':'fgm_30_39''fgm_yds_over_30',
# '40-49 FGM':'fgm_40_49''fgm_yds_over_30',
# '50+ FGM':'fgm_50p''fgm_yds_over_30',
# 'Miss':'fgmiss',
# 'Yds Allowed':'yds_allow_0_100''yds_allow_100_199''yds_allow_200_299''yds_allow_400_449''yds_allow_450_499''yds_allow_500_549''yds_allow_550p',
# 'Pts Agn':'pts_allow_0''pts_allow_1_6''pts_allow_7_13''pts_allow_14_20''pts_allow_21_27''pts_allow_28_34''pts_allow_35p',
# 'Scks':'sack',
# 'DefTD':'def_td',
# 'Safts':'safe',
# ]
# colList = ['Att', 'Comp', 'Pass Yds', 'Pass TDs', 'Int', 'Sck', 'Rush', 'Rsh Yds', 'Rsh TDs', 'Fum', 'Tgt', \
#     'RZ Tgt', 'Rec', 'Rec Yds', 'Rec TDs', 'Kick Ret Yds', 'XPM', 'XPA', 'FGM', 'FGA', '10-19 FGM', '20-29 FGM', \
#         '30-39 FGM', '40-49 FGM', '50+ FGM', 'Miss', 'Yds Allowed', 'Pts Agn', 'Scks', 'DefTD', 'Safts']


In [485]:
user_league['scoring_settings']

{'st_ff': 1.0,
 'pts_allow_7_13': 7.0,
 'def_st_ff': 1.0,
 'rec_yd': 0.1,
 'fum_rec_td': 6.0,
 'bonus_rush_rec_yd_200': 0.0,
 'pts_allow_35p': -2.0,
 'pts_allow_28_34': 0.0,
 'fum': 0.0,
 'pr_yd': 0.1,
 'yds_allow_0_100': 10.0,
 'rush_yd': 0.1,
 'yds_allow_100_199': 7.0,
 'pass_td': 4.0,
 'blk_kick': 2.0,
 'pass_yd': 0.04,
 'safe': 2.0,
 'bonus_rush_yd_200': 0.0,
 'def_td': 6.0,
 'fgm_50p': 0.0,
 'def_st_td': 6.0,
 'bonus_rec_yd_200': 0.0,
 'fum_rec': 2.0,
 'rush_2pt': 2.0,
 'pass_cmp': 0.0,
 'def_3_and_out': 0.0,
 'xpm': 1.0,
 'bonus_pass_yd_300': 0.0,
 'pts_allow_21_27': 2.0,
 'yds_allow_500_549': -5.0,
 'fgm_20_29': 0.0,
 'rush_att': 0.10000000149011612,
 'kr_yd': 0.1,
 'pts_allow_1_6': 8.0,
 'fum_lost': -2.0,
 'def_st_fum_rec': 1.0,
 'int': 2.0,
 'fgm_0_19': 0.0,
 'def_2pt': 2.0,
 'pts_allow_14_20': 5.0,
 'yds_allow_200_299': 4.0,
 'rec': 0.25,
 'ff': 0.0,
 'fgmiss': 0.0,
 'st_fum_rec': 1.0,
 'rec_2pt': 2.0,
 'fgm': 3.0,
 'yds_allow_550p': -5.0,
 'rush_td': 6.0,
 'xpmiss': 0.0,
 'f

# Transform

In [486]:
# Customize predictions for league
# Calculate projected points according to that league's scoring settings
# Multiply by points multiplier
for col in mults.columns[1:]:
    preds[col] = preds[col] * mults.loc[mults['username']==username, col][0]
# Add up point columns
preds['pts_proj'] = preds[mults.columns[1:]].sum(axis=1)
# Add extras for defense starting values
preds.loc[preds['Yds Allowed'].notna(), 'pts_proj'] = preds.loc[preds['Yds Allowed'].notna(), 'pts_proj'] + def_starting_pts
# Add extras for Punt/kick returners
preds.loc[preds['PR']==True, 'pts_proj'] = preds.loc[preds['PR']==True, 'pts_proj'] + (pr_bonus)
preds.loc[preds['KR']==True, 'pts_proj'] = preds.loc[preds['KR']==True, 'pts_proj'] + (kr_bonus)

# Merge in point projections with franchise data
full = preds \
    .merge(rosters, how='left', on='player_id') \
    .merge(owners, how='left', left_on='owner_id', right_on='user_id')

In [487]:
# def getRelProjs(df):
#     # Calculate relative points for each position in each week
#     df['pts_proj'] = df['pts_proj'].fillna(0)
#     df = df.sort_values(by='pts_proj', ascending=False, ignore_index=True)
#     # Loop through weeks and positions
#     for week in df['week'].unique():
#         for pos in rostLims.columns[1:]:
#             # Find floor
#             cond1 = df['position']==pos
#             cond2 = df['week']==week
#             floor = df.loc[cond1&cond2, 'pts_proj'].reset_index(drop=True)[rostLims.loc[rostLims['username']==username, pos][0]]
#             # Find each player's points over floor
#             df.loc[cond1&cond2, 'val_proj'] = df.loc[cond1&cond2, 'pts_proj'] - floor
#     return df

In [488]:
def getRelProjs(df):
    # Calculate relative points for each position in each week
    df['pts_proj'] = df['pts_proj'].fillna(0)
    df = df.sort_values(by='pts_proj', ascending=False, ignore_index=True)
    # Loop through weeks and positions
    for week in df['week'].unique():
        for pos in df['position'].unique():
            # Find floor
            cond_pos = df['position']==pos
            cond_week = df['week']==week
            floor = df.loc[cond_pos&cond_week, 'pts_proj'].min()
            # Find each player's points over floor
            df.loc[cond_pos&cond_week, 'val_proj'] = df.loc[cond_pos&cond_week, 'pts_proj'] - floor
    return df

In [489]:
def getSeasonTotals(df):
    # Sum up pts_proj and val_proj
    totals_proj = df.groupby('player_id')['pts_proj'].sum().reset_index()
    totals_rel = df.groupby('player_id')['val_proj'].sum().reset_index()
    # Merge in totals with player metadata
    unis = df.drop_duplicates(subset='player_id', ignore_index=True).drop(columns=['pts_proj', 'val_proj'])
    df = unis.merge(totals_proj, how='left', on='player_id').merge(totals_rel, how='left', on='player_id')
    return df

In [490]:
def tidyUp(df):
    # Tidy
    df = df[[
        'week', 'player_id', 'pts_proj', 'val_proj', 'franchise_name', 'full_name', 'age', 'years_exp', 'weight',
        'height', 'team', 'position', 'taxi', 'starters', 'players', 'fantasy_positions', 'depth_chart_order',
        'depth_chart_position', 'status', 'injury_status', 'injury_notes',
        'injury_start_date', 'practice_participation', 'practice_description','active'
    ]]
    return df

In [491]:
def findFloors(df, rosterList, number_of_franchises):
    # Find floors for each position to calculate relative projections
    # Roster Dictionary
    rosterDict = {
        'QB':['QB'], 
        'RB':['RB'],
        'WR':['WR'],
        'TE':['TE'],
        'K':['PK'],
        'DEF':['DEF'],
        'FLEX':['RB', 'WR', 'TE'],
        'SUPER_FLEX':['QB', 'RB', 'WR', 'TE'],
    }
    # Sort
    sorted = df.sort_values(by='pts_proj', ignore_index=True, ascending=False)
    # Start selecting the most valuable players in terms of total points for each position
    result = []
    for pos in rosterList:
        # Ignore bench spots
        if pos != "BN":
            # Filter for each position and make sure the player is not already taken
            cond_pos = sorted['position'].isin(rosterDict[pos])
            cond_nottaken = ~sorted['player_id'].isin(result)
            player_filtered = sorted.loc[cond_pos & cond_nottaken]
            # Find highest scoring player that fits
            if len(player_filtered)>0:
                players = player_filtered.head(number_of_franchises)['player_id']
                result.append(players)
    floors = df.loc[df['player_id'].isin(result)]
    floor_qb = floors.loc[floors['position']=='QB']['pts_proj'].min()
    floor_rb = floors.loc[floors['position']=='RB']['pts_proj'].min()
    floor_wr = floors.loc[floors['position']=='WR']['pts_proj'].min()
    floor_te = floors.loc[floors['position']=='TE']['pts_proj'].min()
    floor_pk = floors.loc[floors['position']=='PK']['pts_proj'].min()
    floor_def = floors.loc[floors['position']=='DEF']['pts_proj'].min()
    return floor_qb, floor_rb, floor_wr, floor_te, floor_pk, floor_def

In [492]:
def rosterOptimizer(df):
    # choose a roster for each franchise based on their league's number of available slots
    optimized = pd.DataFrame()
    # Roster Dictionary
    rosterDict = {
        'QB':['QB'], 
        'RB':['RB'],
        'WR':['WR'],
        'TE':['TE'],
        'K':['PK'],
        'DEF':['DEF'],
        'FLEX':['RB', 'WR', 'TE'],
        'SUPER_FLEX':['QB', 'RB', 'WR', 'TE'],
    }
    # Sort
    cond_taxi = df['taxi'].isna()
    sorted = df.loc[cond_taxi].sort_values(by='pts_proj', ignore_index=True, ascending=False)
    # Loop through weeks and franchises
    for this_user in df['user_id'].unique():
        for week_to_optimize in df['week'].unique():
            # Filter predictions for only that week and on a roster
            cond_franch = sorted['user_id']==this_user
            cond_week = sorted['week']==week_to_optimize
            week_filtered = sorted.loc[cond_franch & cond_week]
            # Start selecting the most valuable players in terms of total points for each position
            result = []
            for pos in rosterList:
                # Ignore bench spots
                if pos != "BN":
                    # Filter for each position and make sure the player is not already taken
                    cond_pos = week_filtered['position'].isin(rosterDict[pos])
                    cond_nottaken = ~week_filtered['player_id'].isin(result)
                    player_filtered = week_filtered.loc[cond_pos & cond_nottaken]
                    # Find highest scoring player that fits
                    if len(player_filtered)>0:
                        player = player_filtered.reset_index(drop=True)['player_id'][0]
                        result.append(player)
            # Append results
            next = week_filtered.loc[week_filtered['player_id'].isin(result)]
            optimized = pd.concat([optimized, next], axis=0, ignore_index=True)
    return optimized

# Waiver Report

In [493]:
# Waiver report
waivers = full.copy()
# Calculate relative points for each position in each week
waivers = getRelProjs(waivers)
# Sum points across the season
waivers = getSeasonTotals(waivers)
# Filter for only players on user's roster or unclaimed and who would improve user's points
minpts = waivers.loc[(waivers['display_name']==username)&(waivers['taxi'].isna())]['pts_proj'].min()
cond_user = waivers['display_name']==username
cond_fa = waivers['display_name'].isna()
cond_minpts = waivers['pts_proj']>=minpts
waivers = waivers.loc[(cond_user|cond_fa)&cond_minpts].reset_index(drop=True)
waivers = waivers.sort_values('val_proj', ascending=False, ignore_index=True)
# Tidy
waivers = tidyUp(waivers)

# Roster Optimizer

In [494]:
#from ffpackage.analysis import starterSelector
# Choose a roster for each franchise based on their league's number of available slots
optimized = rosterOptimizer(full)
# Get relative points
optimized = getRelProjs(optimized)
# Sum points across the season
optimized = getSeasonTotals(optimized)
# Visualize
fig = compareFranchises(optimized, how='absolute')
fig.show()

In [495]:
# Swap
playerCC = 'Austin Ekeler'
OPP = 'kevinbash'
playerOPP = 'Amon-Ra St. Brown'
swap = full.copy()
swap.loc[swap['full_name']==playerCC, 'franchise_name'] = OPP
swap.loc[swap['full_name']=='Sam LaPorta', 'franchise_name'] = 'Croccity Body Snatchers'
swap.loc[swap['full_name']==playerOPP, 'franchise_name'] = 'Croccity Body Snatchers'

# Choose a roster for each franchise based on their league's number of available slots
optimized_swap = rosterOptimizer(swap)
# Get relative points
optimized_swap = getRelProjs(optimized_swap)
# Sum points across the season
optimized_swap = getSeasonTotals(optimized_swap)
# Visualize
fig = compareFranchises(optimized_swap, how='absolute')
fig.show()

In [453]:
full.loc[full['full_name']=='Kyren Williams']

,player_id,full_name,birth_date,age,weight,height,team,position,fantasy_positions,depth_chart_order,...,KR,date_prediction,pts_proj,owner_id,taxi,starters,players,user_id,franchise_name,display_name
406,8150,Kyren Williams,2000-08-26,23.0,194.0,69,LAR,RB,['RB'],1.0,...,NaN,20231126,10.435,725907041763303424,NaN,NaN,True,725907041763303424,Croccity Body Snatchers,dirtywizard
407,8150,Kyren Williams,2000-08-26,23.0,194.0,69,LAR,RB,['RB'],1.0,...,NaN,20231126,9.710,725907041763303424,NaN,NaN,True,725907041763303424,Croccity Body Snatchers,dirtywizard
408,8150,Kyren Williams,2000-08-26,23.0,194.0,69,LAR,RB,['RB'],1.0,...,NaN,20231126,10.690,725907041763303424,NaN,NaN,True,725907041763303424,Croccity Body Snatchers,dirtywizard


In [444]:
test = full.groupby(['full_name', 'franchise_name'])['pts_proj'].sum().reset_index()
test = test.loc[(test['franchise_name'].isin(['Pretty Big Wieners'])) | (test['full_name'].isin(['A.J. Brown', 'Amon-Ra St. Brown', 'Brandon Aiyuk', 'Sam LaPorta']))]
test = test[['full_name', 'franchise_name', 'pts_proj']].groupby('full_name').sum()
test.sort_values('pts_proj', ascending=False)

,franchise_name,pts_proj
full_name,,
C.J. Stroud,Pretty Big Wieners,48.9900
A.J. Brown,Pretty Big Wieners,41.0900
Amon-Ra St. Brown,kevinbash,40.3850
Brandon Aiyuk,kevinbash,38.7850
Keenan Allen,Pretty Big Wieners,37.4700
James Cook,Pretty Big Wieners,35.4750
Justin Jefferson,Pretty Big Wieners,33.0150
Alexander Mattison,Pretty Big Wieners,30.7500
Mac Jones,Pretty Big Wieners,30.2500


In [443]:
test = full.groupby(['full_name', 'franchise_name'])['pts_proj'].sum().reset_index()
test = test.loc[(test['franchise_name'].isin(['Croccity Body Snatchers'])) | (test['full_name'].isin(['A.J. Brown', 'Amon-Ra St. Brown', 'Brandon Aiyuk', 'Sam LaPorta']))]
test = test[['full_name', 'franchise_name', 'pts_proj']].groupby('full_name').sum()
test.sort_values('pts_proj', ascending=False)
#  

,franchise_name,pts_proj
full_name,,
Jalen Hurts,Croccity Body Snatchers,71.4100
Brock Purdy,Croccity Body Snatchers,54.0300
Austin Ekeler,Croccity Body Snatchers,51.7000
Kyler Murray,Croccity Body Snatchers,51.3800
Josh Jacobs,Croccity Body Snatchers,51.3050
A.J. Brown,Pretty Big Wieners,41.0900
Joe Mixon,Croccity Body Snatchers,41.0650
Amon-Ra St. Brown,kevinbash,40.3850
Brandon Aiyuk,kevinbash,38.7850


### Left Off Here:
##### Build trade report
##### Tailor scoring for user's unique league
##### Need to figure out how to convert rosterList (number of position slots available) into relative values. Ideas:
 - Standard deviation / Zscore
 - Measure down from the top rather than up from the bottom, 1/(xMax-x)
 - hard-coded numbers of slots
 - percentage of overall points

In [240]:
# floor_qb, floor_rb, floor_wr, floor_te, floor_pk, floor_def = findFloors(full)

In [241]:
rosterList

['QB',
 'RB',
 'RB',
 'WR',
 'WR',
 'WR',
 'TE',
 'TE',
 'K',
 'K',
 'DEF',
 'DEF',
 'FLEX',
 'FLEX',
 'SUPER_FLEX',
 'BN',
 'BN',
 'BN',
 'BN',
 'BN',
 'BN',
 'BN',
 'BN']

In [242]:
countList = {'QB':2*12,
    'RB':5*12,
    'WR':6*12,
    'TE':5*12,
}

countListPartial = {
'QB':int(round(1.25*12, 0)),
'RB':int(round(2.9*12, 0)),
'WR':int(round(3.9*12, 0)),
'TE':int(round(2.9*12, 0)),}

In [243]:
full['pts_proj'].sum()

19975.5847

In [244]:
test = full.copy()
test = getRelProjs(test)
test = getSeasonTotals(test)
test = test.sort_values(by='pts_proj', ignore_index=True, ascending=False)
cond_pospts = test['pts_proj']>0

for pos in ['QB', 'RB', 'WR', 'TE']:
    print(f'{pos} Mean: ' + str(test.loc[(test['position']==pos) & cond_pospts]['pts_proj'].mean().round(2)))
    print(f'{pos} Median: ' + str(test.loc[(test['position']==pos) & cond_pospts]['pts_proj'].median().round(2)))
    print(f'{pos} Spot #{countListPartial[pos]}: ' + str(test.loc[(test['position']==pos) & cond_pospts].reset_index(drop=True).iloc[countListPartial[pos]]['pts_proj']))
    print(f'{pos} Spot #{countList[pos]}: ' + str(test.loc[(test['position']==pos) & cond_pospts].reset_index(drop=True).iloc[countList[pos]]['pts_proj']))


QB Mean: 58.21
QB Median: 57.09
QB Spot #15: 105.91
QB Spot #24: 83.23
RB Mean: 47.99
RB Median: 41.39
RB Spot #35: 58.835
RB Spot #60: 28.175
WR Mean: 40.61
WR Median: 34.03
WR Spot #47: 52.285000000000004
WR Spot #72: 34.592000000000006
TE Mean: 22.68
TE Median: 18.14
TE Spot #35: 22.355
TE Spot #60: 11.33


In [245]:
# for pos in ['QB', 'RB', 'WR', 'TE', 'PK', 'DEF']:
    

# Find trade targets

In [246]:
# chop_df = full.copy()
# # Type player's name to find their vitals
# chop = 'Zay Jones'
# # See how bold the user wants to be
# multiplier_tradeup = 1.05
# # Calc across season
# # Get relative points
# chop_df = getRelProjs(chop_df)
# # Sum points across the season
# #chop_df = getSeasonTotals(chop_df)
# chop_df


In [247]:
# # Find the player
# chop_row = chop_df.loc[chop_df['full_name']==chop].head(1).reset_index().iloc[0]
# #chop_id = chop_row['player_id']
# chop_pos = chop_row['position']
# chop_age = chop_row['age']
# chop_rel = chop_row['val_proj']
# # Select
# #cond_id = chop_df['player_id']!=chop_id
# cond_rel_min = chop_df['val_proj']>=chop_rel
# cond_rel_max = chop_df['val_proj']<=chop_rel * multiplier_tradeup
# cond_notfa = chop_df['franchise_name'].notna()
# targs = chop_df.loc[cond_rel_min & cond_rel_max & cond_notfa]
# targs

# Write to CSV

In [248]:
# sums.to_csv(f'processed_{date_string}.csv', index=False)
waivers.to_csv(f'data_sharks/waivers.csv', index=False)

In [168]:
# # Starter report
# full.loc[full['starters']==True].groupby('franchise_name')['rel_pts'].sum().sort_values()

In [169]:
full.loc[full['starters']==True].groupby('franchise_name')['pts_proj'].sum().sort_values(ascending=False)

franchise_name
Saskatoon Squatches         172.8338
kevinbash                   155.9903
2014champ                   150.3302
Croccity Body Snatchers     140.0832
Comeback 👑’s                129.0672
Pretty Big Wieners          127.1324
Verdanks Vacqueros FFC      123.7279
DangeRUSS Last Ride         121.4686
*New Name Pending*          120.0930
GusTheBus                   108.7678
Idk much about soccer        86.1842
📜 Providence 🪬 Spirits 🥂     84.5920
Name: pts_proj, dtype: float64

In [379]:
full.columns

Index(['player_id', 'full_name', 'birth_date', 'age', 'weight', 'height',
       'team', 'position', 'fantasy_positions', 'depth_chart_order',
       'depth_chart_position', 'status', 'injury_status', 'injury_notes',
       'injury_start_date', 'practice_participation', 'practice_description',
       'years_exp', 'active', 'id_sharks', 'id_ourlads', 'week', 'Att', 'Comp',
       'Pass Yds', 'Pass TDs', 'Int', 'Sck', 'Rush', 'Rsh Yds', 'Rsh TDs',
       'Fum', 'Tgt', 'RZ Tgt', 'Rec', 'Rec Yds', 'Rec TDs', 'Kick Ret Yds',
       'XPM', 'XPA', 'FGM', 'FGA', '10-19 FGM', '20-29 FGM', '30-39 FGM',
       '40-49 FGM', '50+ FGM', 'Miss', 'Yds Allowed', 'Pts Agn', 'Scks',
       'DefTD', 'Safts', 'PR', 'KR', 'date_prediction', 'pts_proj', 'owner_id',
       'taxi', 'starters', 'players', 'user_id', 'franchise_name',
       'display_name'],
      dtype='object')

In [403]:
test = optimized.loc[(optimized['franchise_name'].isin(['Croccity Body Snatchers', 'Pretty Big Wieners']))]
test = test.groupby('full_name')['pts_proj'].sum()
test.sort_values(ascending=False)

full_name
Jalen Hurts           23.4800
Kyler Murray          18.7900
Josh Jacobs           16.1950
Austin Ekeler         15.6800
A.J. Brown            15.0000
David Montgomery      14.2200
James Cook            14.1000
Joe Mixon             13.9950
Keenan Allen          12.5150
Dustin Hopkins        11.7500
Alexander Mattison    11.2150
Mac Jones             11.1800
Tyler Lockett         10.7600
Justin Jefferson      10.4750
Chris Godwin           9.9950
Greg Zuerlein          9.7000
Cairo Santos           9.0500
Daniel Carlson         9.0000
MIA                    8.4712
Rashee Rice            8.3800
D'Onta Foreman         8.1900
Miles Sanders          7.5050
Robert Woods           6.2280
Tyler Conklin          4.8470
Tyler Higbee           4.5100
Daniel Bellinger       4.4900
SF                     3.4448
PHI                    2.1330
Adam Trautman          1.9300
DEN                   -0.5460
Name: pts_proj, dtype: float64